In [46]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

train_data=pd.read_csv("train.csv")
train_data.head()

test_data=pd.read_csv("test.csv")
test_data.head()



X_train = train_data.drop(['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'], axis=1)
y_train = train_data[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]


X_train.head()

y_train.head()

test_ids=test_data['id']
test_features= test_data.drop("id",axis=1)

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier 
from sklearn.multioutput import MultiOutputClassifier 
from lightgbm import LGBMClassifier 
from xgboost import XGBClassifier 
from catboost import CatBoostClassifier 
import joblib

rf_classifier=RandomForestClassifier()
lgbm_classifier=LGBMClassifier()
xgb_classifier=XGBClassifier()
catboost_classifier=CatBoostClassifier()
extratrees_classifier = ExtraTreesClassifier()

rf_multi_classifier = MultiOutputClassifier(rf_classifier, n_jobs=-1)
lgbm_multi_classifier = MultiOutputClassifier(lgbm_classifier, n_jobs=-1)
xgb_multi_classifier = MultiOutputClassifier(xgb_classifier, n_jobs=-1)
catboost_multi_classifier = MultiOutputClassifier(catboost_classifier, n_jobs=-1)
extratrees_multi_classifier = MultiOutputClassifier(extratrees_classifier, n_jobs=-1)

rf_cv_scores = cross_val_score(rf_multi_classifier, X_train, y_train,cv=5, scoring='accuracy')
lgbm_cv_scores = cross_val_score(lgbm_multi_classifier, X_train, y_train,cv=5, scoring='accuracy')
xgb_cv_scores = cross_val_score (xgb_multi_classifier, X_train, y_train,cv=5, scoring='accuracy')
catboost_cv_scores = cross_val_score(catboost_multi_classifier, X_train, y_train,cv=5, scoring='accuracy')
extratrees_cv_scores = cross_val_score(extratrees_multi_classifier, X_train, y_train,cv=5, scoring='accuracy')

print( "Random Forest CV Accuracy: ",rf_cv_scores.mean ())
print("LightGBM CV Accuracy:", lgbm_cv_scores.mean())
print( "XBoost CV Accuracy: ", xgb_cv_scores.mean ())
print( "CatBoost CV Accuracy:", catboost_cv_scores. mean()) 
print("ExtraTrees CV Accuracy:", extratrees_cv_scores.mean ())

best_model = max([
        (rf_cv_scores.mean(), 'Random Forest'),
        (lgbm_cv_scores.mean(), 'LightGBM'),
        (xgb_cv_scores.mean (), 'XGBoost'),
        (catboost_cv_scores.mean(), 'CatBoost'),
        (extratrees_cv_scores.mean(), 'ExtraTrees')

])
print("best model:",best_model[1])

# Print best_model[1] to check its value
print("Best model:", best_model[1])

best_classifier = None
if best_model[1] == 'Random Forest':
    best_classifier = rf_multi_classifier.fit(X_train, y_train)       
elif best_model[1] == 'LightGBM':
    best_classifier = lgbm_multi_classifier.fit(X_train, y_train)
elif best_model[1] == 'XGBoost':
    best_classifier = xgb_multi_classifier.fit(X_train, y_train)
elif best_model[1] == 'CatBoost':
    best_classifier = catboost_multi_classifier.fit(X_train, y_train)
elif best_model[1] == 'ExtraTrees':
    best_classifier = extratrees_multi_classifier.fit(X_train, y_train)

# Check if best_classifier is still None
if best_classifier is None:
    print("Warning: No classifier was assigned for the selected model.")
else:
    # Now you can use best_classifier for predictions
    best_test_probs = best_classifier.predict_proba(test_features)


model=joblib.dump(best_classifier,'best_model.joblib')

best_test_probs = best_classifier.predict_proba(test_features)

print(best_test_probs)

import time
submission_df=pd.DataFrame({'id': test_ids})
for i, target in enumerate(y_train.columns):
    submission_df[target]=best_test_probs[i][:,1]
    time.sleep(3)
    print(submission_df)


submission_df.to_csv('submission.csv',index=False)

[LightGBM] [Info] Number of positive: 2741, number of negative: 12634
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007337 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5201
[LightGBM] [Info] Number of data points in the train set: 15375, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.178276 -> initscore=-1.528069
[LightGBM] [Info] Start training from score -1.528069
[LightGBM] [Info] Number of positive: 906, number of negative: 14469
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005529 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5205
[LightGBM] [Info] Number of data points in the train set: 15375, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058927 -> initscore=-2.770724
[Ligh